#Sentiment Analysis on Movie Reviews
Author: Lufei Ouyang

Data set: Pang and Lee Movie Review Data

In [ ]:
from torchtext.vocab import Vectors
from torchtext.vocab import GloVe
import torch
import pandas as pd
import numpy as np

### Load the word vector for generating the embedding matrix

In [ ]:
vec = GloVe(name='6B', dim=300)

### Read trainning data

In [ ]:
import os
#read trainning data
neg_train_files = os.listdir('neg')
pos_train_files = os.listdir('pos')

pos_train_string = ''
neg_train_string = ''
f_train_pos = None
f_train_neg = None
i = 0
for file in pos_train_files:
  if "txt" in file and int(file[2:5]) < 600:
        i+=1
        f_train_pos=open(os.path.join('pos',file),'r')
        pos_train_string += f_train_pos.read()
        f_train_pos.close()
for file in neg_train_files:
  if "txt" in file and int(file[2:5]) < 600:
        f_train_neg=open(os.path.join('neg',file),'r')
        neg_train_string += f_train_neg.read()
        f_train_neg.close()
len(pos_train_files)

1000

### Read test data

In [ ]:
#read test data
neg_test_files = os.listdir('neg')
pos_test_files = os.listdir('pos')
pos_test_string = ''
neg_test_string = ''
f_test_pos = None
f_test_neg = None
i = 0
for file in pos_test_files:
  if "txt" in file and int(file[2:5]) >= 600:
        
        f_test_pos=open(os.path.join('pos',file),'r')
        pos_test_string += f_test_pos.read()
        f_test_pos.close()
for file in neg_test_files:
  if "txt" in file and int(file[2:5]) >= 600:
        i+=1
        f_test_neg=open(os.path.join('neg',file),'r')
        neg_test_string += f_test_neg.read()
        f_test_neg.close()
len(neg_train_files)

1001

### Preprocessing the data

In [ ]:
#processing train data

#split each sentence
pos_train_sentences = pos_train_string.split(' \n')
pos_train_words = []
pos_train_glove = []

#split each word into a list
for sentence in pos_train_sentences:
  pos_train_words.append(sentence.split(' '))
for sentence_words in pos_train_words:
  pos_train_glove.append(vec.get_vecs_by_tokens(sentence_words, lower_case_backup=True))
#split each sentence
neg_train_sentences = neg_train_string.split(' \n')
neg_train_words = []
neg_train_glove = []
#split each word into a list
for sentence in neg_train_sentences:
  neg_train_words.append(sentence.split(' '))
for sentence_words in neg_train_words:
  neg_train_glove.append(vec.get_vecs_by_tokens(sentence_words, lower_case_backup=True))


In [ ]:
#processing test data

#split each sentence
pos_test_sentences = pos_test_string.split(' \n')
pos_test_words = []
pos_test_glove = []
#split each word into a list
for sentence in pos_test_sentences:
  pos_test_words.append(sentence.split(' '))
for sentence_words in pos_test_words:
  pos_test_glove.append(vec.get_vecs_by_tokens(sentence_words, lower_case_backup=True))
#split each sentence
neg_test_sentences = neg_test_string.split(' \n')
neg_test_words = []
neg_test_glove = []
#split each word into a list
for sentence in neg_test_sentences:
  neg_test_words.append(sentence.split(' '))
for sentence_words in neg_test_words:
  neg_test_glove.append(vec.get_vecs_by_tokens(sentence_words, lower_case_backup=True))
#create the label list
label_test = []
sentence_test_vec = []
for i in range(len(pos_test_glove)):
  sentence_test_vec.append(pos_test_glove[i])
  label_test.append(torch.IntTensor(1))
for i in range(len(neg_test_glove)):
  sentence_test_vec.append(neg_test_glove[i])
  label_test.append(torch.IntTensor(0))

### Convert data into sequence and embedding matrix to make it usable in neural network

In [ ]:
#convert list of train data to sequence
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000000)
tokenizer.fit_on_texts(sentence_train)
word_index = tokenizer.word_index
train_sequences = tokenizer.texts_to_sequences(sentence_train)
data = pad_sequences(train_sequences, maxlen=100)


In [ ]:
#convert list of test data to sequence
sentence_test =pos_test_sentences + neg_test_sentences
test_sequences = tokenizer.texts_to_sequences(sentence_test)
test_data = pad_sequences(test_sequences, maxlen=100)

In [ ]:
#create train label data 
label_train = np.zeros(len(sentence_train))

for i in range(len(pos_train_sentences)):
  label_train[i] = 1
 

In [ ]:
#create test label data 
label_test = np.zeros(len(sentence_test))
for i in range(len(pos_test_sentences)):
  label_test[i] = 1

In [ ]:
#create embedding matrix
sentence_train_vec = np.zeros((len(word_index) + 1, 300))

for word, i in word_index.items():
  sentence_train_vec[i] = vec.get_vecs_by_tokens(word, lower_case_backup=True)

###Create neural network layers

In [ ]:
#create model and train model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers import Dense
from keras.layers import MaxPool1D
import keras
model = Sequential()
model.add(Embedding(len(word_index) + 1,300,weights=[sentence_train_vec],input_length=150,trainable=True))


model.add(LSTM(128, return_sequences=True, input_shape=(50, 150)))
model.add(MaxPool1D(pool_size=150))
model.add(Flatten())

model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#train the neural network
model.fit(data,label_train,epochs=20,batch_size =100 , verbose=1)

Epoch 1/20
384/384 [==============================] - 48s 121ms/step - loss: 0.6549 - accuracy: 0.6025
Epoch 2/20
384/384 [==============================] - 46s 121ms/step - loss: 0.4783 - accuracy: 0.7743
Epoch 3/20
384/384 [==============================] - 46s 121ms/step - loss: 0.3181 - accuracy: 0.8578
Epoch 4/20
384/384 [==============================] - 47s 122ms/step - loss: 0.2215 - accuracy: 0.9047
Epoch 5/20
384/384 [==============================] - 46s 121ms/step - loss: 0.1619 - accuracy: 0.9304
Epoch 6/20
384/384 [==============================] - 47s 121ms/step - loss: 0.1188 - accuracy: 0.9510
Epoch 7/20
384/384 [==============================] - 46s 121ms/step - loss: 0.0912 - accuracy: 0.9629
Epoch 8/20
384/384 [==============================] - 46s 120ms/step - loss: 0.0663 - accuracy: 0.9752
Epoch 9/20
384/384 [==============================] - 47s 121ms/step - loss: 0.0494 - accuracy: 0.9818
Epoch 10/20
384/384 [==============================] - 46s 120ms/step - l

### Summary of the neural network model

In [ ]:
model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 150, 300)          10339800  
_________________________________________________________________
lstm_8 (LSTM)                (None, 150, 128)          219648    
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 1, 128)            0         
_________________________________________________________________
flatten_8 (Flatten)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 10,559,577
Trainable params: 10,559,577
Non-trainable params: 0
_________________________________________________________________


### Evaluate the neural network by using test set

In [ ]:
#result
model.evaluate(test_data, label_test)

824/824 [==============================] - 5s 6ms/step - loss: 3.4709 - accuracy: 0.6057


[3.47090744972229, 0.6056787371635437]

#Project Summary

I use the original Pang and Lee set to make my data set. Since Pang and Lee set does not provide test data set, I split the data set into a trainning data set and a test data set.

I chose to use torchtext's glove 6d dim300 to generate the embedding matrix. In the part of building my model, I chose to use Keras to build the model. The reason is that the Sequential class of Keras is the easiest way I have found to build a new model. I just need to add different layers and train this model. My layers are embedding layer -> LSTM layer -> max-pooling -> Flatten layer -> Dense layer.

First, I read the training data into a list, and then divide it into sentences. Then use keras.preprocessing to add the word vector of each different word to the embedding matrix. Then padding data set. Because we only need to decide whether a sentence is positive or negative. So, I use 0 for negative and 1 for positive to create a label list. Finally, I use the embedding matrix to create the embedding layer and use the sequence of the sentence as the data, and the list containing 0, 1 as the label list for training. After 20 iterations, the accuracy of the model can reach 99.38%. But the accuracy of test data is only 60.57%. 

I think the reason is that there are many sentences acting as noise components(each review is positive or negative, but not every sentence showing the positive or negative attitude), so the accuracy of the test set is so low.
